In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
from gym import Env, spaces
from qpython import qconnection
import numpy as np
import json

In [9]:
def _exec(qry, pandas=False):
        with qconnection.QConnection(
            host="localhost", 
            port=5050, 
            pandas=pandas) as q:
            return q.sendSync(qry) 

In [17]:
data = _exec("getdata[`events;30]", pandas=True)

In [20]:
data["cmd"]=(data["cmd"]).apply(lambda x:x.decode("utf-8"))
data["kind"]=(data["kind"]).apply(lambda x:x.decode("utf-8"))

In [21]:
data

,date,time,intime,kind,cmd,datum
0,2020-07-26,2020-07-26 23:54:24.490,2020-07-26 23:54:24.547,TRADE,NEW,"[b'SELL', 993500, 1]"
1,2020-07-26,2020-07-26 23:54:44.650,2020-07-26 23:54:44.708,TRADE,NEW,"[b'BUY', 993550, 92]"
2,2020-07-26,2020-07-26 23:54:44.650,2020-07-26 23:54:44.708,TRADE,NEW,"[b'BUY', 993550, 110]"
3,2020-07-26,2020-07-26 23:54:44.650,2020-07-26 23:54:44.708,TRADE,NEW,"[b'BUY', 993550, 200]"
4,2020-07-26,2020-07-26 23:54:44.650,2020-07-26 23:54:44.708,TRADE,NEW,"[b'BUY', 993550, 4]"
...,...,...,...,...,...,...
180239,2020-07-27,2020-07-27 00:08:00.000,2020-07-27 00:08:00.771,MARK,UPDATE,[[995552]]
180240,2020-07-27,2020-07-27 00:08:05.132,2020-07-27 00:08:05.786,MARK,UPDATE,[[995562]]
180241,2020-07-27,2020-07-27 00:08:15.360,2020-07-27 00:08:15.759,MARK,UPDATE,[[995484]]
180242,2020-07-27,2020-07-27 00:08:30.080,2020-07-27 00:08:30.736,MARK,UPDATE,[[995349]]
